In [3]:
import rasterio
from rasterio.features import shapes
import json



In [2]:

# Path to the DEM file
dem_path = "/Users/tomassalazar/Desktop/WUSTL/School/Research Geology UC- FA /RV HYP MAP/Rapaki Misc Data/mosaic_dem.tif"

# Output GeoJSON file
geojson_path = "RVDEM.geojson"



In [3]:

# Open the DEM
with rasterio.open(dem_path) as src:
    # Read the raster data
    raster = src.read(1)  # Reading the first band
    transform = src.transform  # Affine transformation

    # Extract shapes and values
    shapes_and_values = shapes(raster, transform=transform)

    # Convert shapes and values into GeoJSON format
    features = []
    for geom, value in shapes_and_values:
        features.append({
            "type": "Feature",
            "geometry": geom,
            "properties": {"elevation": value}
        })

    # Create a GeoJSON structure
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

In [ ]:
# Write GeoJSON to a file
with open(geojson_path, "w") as f:
    json.dump(geojson, f)

print(f"GeoJSON file saved to {geojson_path}")

In [1]:
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point

In [4]:
ds = gdal.Open(dem_path)
band = ds.GetRasterBand(1)  # Get the first band
transform = ds.GetGeoTransform()  # Affine transform
data = band.ReadAsArray()  # Read the raster as a numpy array

# Get raster dimensions
cols = ds.RasterXSize
rows = ds.RasterYSize

# Convert raster to a GeoDataFrame of points
points = []
for row in range(rows):
    for col in range(cols):
        x = transform[0] + col * transform[1] + row * transform[2]
        y = transform[3] + col * transform[4] + row * transform[5]
        value = data[row, col]
        if value != band.GetNoDataValue():  # Exclude NoData values
            points.append({"geometry": Point(x, y), "elevation": value})

gdf = gpd.GeoDataFrame(points, crs="EPSG:4326")  # Update CRS if needed

# Save to GeoJSON
gdf.to_file(geojson_path, driver="GeoJSON")

print(f"GeoJSON saved to {geojson_path}")

GeoJSON saved to RVDEM.geojson
